# extracting line, paragraph, and page level ocr data

https://en.wikipedia.org/wiki/HOCR

In [1]:
import bs4
soup = bs4.BeautifulSoup(Path("/Users/tonyfast/Downloads/fashions-handee_hocr.html").read_text(), features="html.parser")

load ocr data as an array

In [2]:
elements = Series(soup.body.find_all(), name="element")

tidy the data

In [3]:
df = elements.apply(
    compose_left(operator.attrgetter("attrs"), Series)
).join(elements)

classes = (
    df["class"]
    .dropna()
    .explode()
    .str.removeprefix("ocr_")
    .str.removeprefix("ocrx_")
)

assert len(classes.index.unique()) == len(classes)

df = df.drop(columns="class").join(classes.rename("klass"))
df = df.assign(
    page=df.klass.eq("page").cumsum(),
    par=df.klass.eq("par").cumsum(),
    word=df.klass.eq("word").cumsum(),
    line=df.klass.eq("line").cumsum()
)
page_line = pl = df.groupby("page").line.apply(lambda s: s - s.min())
df = df.assign(page_line = Series(pl.values, pl.index.get_level_values(1), name="page_line"))
df = df.join(df[(df.klass == "word")].element.apply(bs4.Tag.get_text).rename("letters"))

combine the words on each line

In [4]:
lines = df.groupby("page par page_line".split()).letters.agg(
    compose(lambda x: x or None, " ".join, Series.dropna)
).dropna()

aggregate the page content

In [5]:
    (
        pages := lines
            .groupby("page par".split())
            .agg("\n".join)
            .groupby("page")
            .agg("\n\n".join)
    )
    for page, text in pages.items(): display(Markdown(F"### Page {page}")), print(text)

### Page 1

FASHIONS FOR THE HANDICAPPED

CUSTOM MADE GARMENTS
or
SEW—IT—YOURSELF KITS

WOMEN’S WASH & WEAR, REASONABLY PRICED
DESIGNED FOR COMFORT AND CONVENIENCE
OF THE HANDICAPPED

C.O. Smith
7674 Park Ave.
Lowville, N.Y. 13367

Telephone (315) 376-7020


### Page 2

PICK A POCKET

Walker Pocket + 61

Two sturdy pockets that fasten securely to
walker bar with 3 Velcro tabs. Provides lots of
carrying space. Sturdy construction and hold
fast stitching. :

onesize 12x15" $4.50

Also available with expansion pockets, more
carrying space.

#61X $5.00

Double Walker Pockets + 65

Twice the pockets of # 61. Two pockets hang
on front and back of walker bar, secured by
straps with Velcro.

$7.00

Saddlebag + 62

This crutch pocket fastens to the hand bar of
crutches with two sets of ties. Tab on back
keeps pocket from swinging out. Slip on from
bottom. May also work on some canes.

Regular size, 8’ x 11”
$3.00

Large size, 10%" x 12”
$3.50

Catheter Pocket + 66 (not shown)

An envelope type pocket to hide drainage
bag. This can be ordered to match any
garment in the HANDEE line. No charge with
order. This is complimentary to anyone who
needs it. Size 12” x 14”

Side Saddle Bag # 69

Two expansion type pockets tie on the arm of
a wheelchair. Sturdy construct

### Page 3

...

DRESSES

Three Armhole Dress = 22

Even a person with the most difficult dressing
problems can manage this wrap dress. The
third armhole anchors the dress, no closures
are necessary. Large armholes, with or
without sleeves. Patch pocket. Finished in
contrasting fold-over braid.

Shirred Bodice Back Wrap = 27

This deep V back wrap is sleeveless. Self tie
belt, in-seam pocket. This stunning dress,
finished in contrasting braid, will take you
through all seasons with the addition of a
bolero or long-line jacket with sleeves.

Sizes 10-18 18% - 22%

Knit or woven polyester seersucker

#27 # 27 Kit

sleeveless $28.00 $14.00
with sleeves $32.00 $16.00
ensemble with jacket $38.00 $19.00

small medium large

knit or woven polyester prints
* cotton-dacron-poly prints

# 22 # 22 Kit
$24.00 $12.00
$28.00 $16.00

sleeveless
with sleeves


### Page 12

Split Second Dress # 25

‘ This carefree, easy-on style with front zipper

: is styled after the Yoked Caftan # -45 The
same roomy sleeves and shoulders, with two
choices:

(1) Closed back.

(2) Split skirt back. The back opens from
yoke to hem with ample overlap. No
closures to sit on.

Choice of neckline, round or V, with lace.
In-seam pocket.

small medium large

polyester knit, velour

# 25 Kit $14.00

# 25
$28.00

# 45

fate

ae RS

—

icing inne oc

DRESS — CAFTAN

Back and Front # 25

Yoked Caftan # 45

Styled also as Split-Second Dress # 25. Split
skirt on side or not. Front opens at yoke with
Velcro tab, in-seam pocket. Bracelet sleeves.
See above.
small

medium large

nylon jersey polyester prints

seersucker
# 45 # 45 Kit
$26.00 $13.00
velour —
$30.00 $15.00

[12]


### Page 13

JUMPERS

Flared Skirt Jumper # 56

A roomy style with bias skirt. In-seam pocket.

Sizes 8 - 18
= ‘
Polyester knit in green, pink i

blue or yellow solids or prints |

Calcutta cloth |

#56 # 56 Kit
$28.00 $14.00

Front Wrap Jumper # 57

This is a popular front wrap style secured by
Velcro on overlap. Generous armholes,
pocket, self-tie belt.

Sizes 8- 18 #57

denim, velour, polyester knits
kettle cloth, Calcutta cloth

#57 # 57 Kit
$28.00 $14.00

Princess Jumper # 59 |

Our open back jumper has a choice of two
closures: :

(1) zipped for easier dressing
(2) full back opening to hem with Velcro tabs
closing to waist.

Sizes 8- 18

# 59 denim polyester knit kettle cloth
Calcutta cloth

# 59 # 59 Kit
$28.00 $14.00
